__качаем объявления о продаже квартир с циан.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
import sys
import re
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import random

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [3]:
raw_data_path = 'data/raw/flat/cian' # каталог с файлами объявлений

In [4]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'
# base_url, req_param = 'https://sevastopol.cian.ru/kupit-kvartiru/', ''
place = 'sevastopol'
base_url, req_param ='https://www.cian.ru/cat.php', 'deal_type=sale&offer_type=flat&region=184723'

In [5]:
%%time

# from lib.downloader import DownloaderSimple
# from lib.parser import AdsListParser
from lib.downloader import DownloaderSeleniumFirefox
from lib.cian import CianParser


df,html = CianParser(
        base_url = base_url,
        driver=DownloaderSeleniumFirefox(profile_path),
        # driver=DownloaderSimple(),
    ).load(req_param=req_param, keep_html=True) # , npage_start=53) # , page_limit=3 ) #

print(len(df))

[INFO    ] 2022-11-15 13:06:57 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-11-15 13:06:57 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-11-15 13:07:00 | AdsListParser: downloader init
[INFO    ] 2022-11-15 13:07:00 | CianParser: downloader init
[INFO    ] 2022-11-15 13:07:00 | AdsListParser: start read and parse pages...
[INFO    ] 2022-11-15 13:07:13 | AdsListParser: read page 1
[INFO    ] 2022-11-15 13:07:22 | AdsListParser: read page 2
[INFO    ] 2022-11-15 13:07:34 | AdsListParser: read page 3
[INFO    ] 2022-11-15 13:07:44 | AdsListParser: read page 4
[INFO    ] 2022-11-15 13:07:53 | AdsListParser: read page 5
[INFO    ] 2022-11-15 13:08:09 | AdsListParser: read page 6
[INFO    ] 2022-11-15 13:08:20 | AdsListParser: read page 7
[INFO    ] 2022-11-15 13:08:29 | AdsListParser: read page 8
[INFO    ] 2022-11-15 13:08:42 | AdsListParser: read page 9
[INFO    ] 2022-11-15 13:08:55 | AdsListParser: read page 10
[INFO    ] 2022-11-15 13:09:06 | AdsLis

In [6]:
df.sample(3)

,OfferTitle,OfferSubtitle,Deadline,MainPrice,PriceInfo,GeoLabel,TimeLabel,LinkArea,Description,page,ts
1366,"3-комн. кв., 89,1 м², 6/8 этаж",,,16 900 000 ₽,189 675 ₽/м²,"[Севастополь, р-н Гагаринский, мкр. Камышовая бухта, проспект Героев Сталинграда, 53]","[пять дней назад, 9 ноя, 16:25]",https://sevastopol.cian.ru/sale/flat/275426935/,"Трехкомнатная квартира 89,1 м2, 6/8 эт. \nТрехкомнатная квартира 89,1 м2 с кухней 15 м2 на 6/8 этаже дома. Расположена в г. Севастополь, Гагаринский район, пр-т героев Сталинграда, д.53. Прекрасное предложение для дружной семьи или в качестве высокорентабельной инвестиции. \nПозвоните нам, чтобы узнать все детали! \nТехнические характеристики: \n \n-Адрес: г. Севастополь, Гагаринский район, пр-т героев Сталинграда д. 53\n-Год постройки дома: 2005 г.\n-Этаж/этажность: 6/8\n-Количество комнат: 3\n-Комнаты: изолированные\n-Общая площадь: 89,1 м2\n-Жилая площадь: 50 м2\n-Площадь кухни: 15 м2\n-Высота потолка: 2,8 м\n-Санузел: совмещенный\n-Водоснабжение: центральное\n-Горячее водоснабжение: АГВ\n-Отопление: АГВ\nДополнительная информация: \nКвартира расположена на шестом этаже восьмиэтажного дома, построенного из инкерманского блока - природный материал, экологически чистый, прекрасно держит температуру( не нагревается летом и не успевает остывать зимой). \nВ квартире выполнен ремонт с использованием качественных строительных материалов. Общая площадь квартиры составляет 89,1 кв.метров. Кухня-гостиная и две изолированные комнаты, два сан.узла, в одном из которых оборудован Хамам. Квартира укомплектована всем необходимым для комфортного проживания. Вся мебель и техника останется новым собственникам. \nВ квартире установлена умная система приточной вентиляции Тион (очистка воздуха от пыли, аллергенов, загрязнений воздуха с улицы, в том числе выхлопных газов). Есть возможность подогрева воздуха и дистанционного управления системой.\nТерритория дома огорожена, имеются входы и въезды с разных сторон двора. Также есть огороженные детская и спортивная площадки, только для жильцов этого дома. Всегда есть возможность припарковать автомобиль, что очень актуально на сегодняшний день.\nПозвоните нам, чтобы назначить просмотр! \nИнфраструктура: \nРасположение квартиры очень удобное. В шаговой доступности несколько городских пляжей: ""Омега"", ""Парк Победы"", Солдатский пляж. При этом вся социально значимая инфраструктура также в шаговой доступности., детские сады 129, 123, 130, Школы 15, 54. Инженерная школа, множество центров доп.образования для детей и взрослых, магазины, рынки, аптеки. Прекрасная транспортная развязка, остановка общественного транспорта в 3 минутах ходьбы, от куда без труда можно уехать абсолютно в любую точку города.\nПодходит для любых видов расчета.\nДокументы проверены юристами и готовы к сделке. \nПрофессиональное сопровождение до получения права собственности. \nДобавьте предложение в закладки, чтобы не потерять!",50,2022-11-15 13:16:52.865293
864,"3-комн. кв., 57,3 м², 2/5 этаж",,,6 300 000 ₽,109 948 ₽/м²,"[Севастополь, р-н Ленинский, мкр. Острякова, улица Маршала Геловани, 4]","[пять дней назад, 9 ноя, 16:17]",https://sevastopol.cian.ru/sale/flat/274384418/,"Продаётся трехкомнатная квартира на улице Маршала Геловани д.4!\nОтличное предложение для проживания или в качестве высокорентабельной инвестиции. Квартира отлично подойдет для большой и дружной семьи!\nРассмотрим покупателей с наличным расчетом, ипотекой, материнским капиталом и жилищными сертификатами.\nКвартира расположена в тихом, зелёном районе города, рядом со всей необходимой инфраструктурой для комфортной жизни, на 2 этаже 5-ти этажного дома из инкерманского камня.\nОбщая площадь квартиры 57.3 кв.м , преимуществом данной квартиры является ее планировка. Комнаты изолированные просторные и светлые площадью 17. 5, 11.3 и 10.9 кв.м. с выходом на балкон площадью 3.9 кв.м, который можно оборудовать как дополнительную комнату. Высота потолков 2.65 м.\nКухня площадью 5.2 кв.м, правильной формы, что позволяет распределить рабочу

In [7]:
print(len(df))
df = pd.DataFrame(df).dropna().drop_duplicates(['LinkArea','Description',]).reset_index(drop=True)
print(len(df))

1496
1437


In [8]:
df['place']=place

In [9]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
# df.to_excel(f'{raw_data_path}/cian_{ts}_raw.xlsx',index=False)
df.to_pickle(f'{raw_data_path}/cian_{ts}_raw.pkl')

----

In [10]:
# df = pd.read_excel('data/raw/cian/cian_2022-09-08_13-49_raw.xlsx')

In [11]:
# df

In [12]:
# df['GeoLabel'].apply( lambda s : len(s) ) #', '.join(s) )

# s = ['Севастополь', 'р-н Гагаринский', 'Казачья Бухта мкр', 'Скифия ЖК']
# ', '.join(s)

In [13]:
# from lib.cian import CianDataCleaner

# CianDataCleaner().transform(df)

----

In [14]:
# with open('tmp/cian.html','wt') as f: f.write(html[0])